In [ ]:
import time 
import torch
import matplotlib.pyplot as plt
import matplotlib
#from matplotlib import rc
import numpy as np

from one_d_excited_states import *
import matrix_element as me

from os import path
import os
from copy import copy


In [ ]:
use_gpu = False

# Check to see if gpu is available. If it is, use it else use the cpu
if torch.cuda.is_available() and use_gpu:
    device = torch.device('cuda')
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print('Using GPU.')
else:
    device = torch.device('cpu')
    torch.set_default_tensor_type('torch.DoubleTensor')

    if not torch.cuda.is_available() and use_gpu: 
        use_gpu = False 
        print('GPU not available. Using CPU.')
    else: 
        print('Using CPU.')


In [ ]:
N_list = [7]
N = N_list[0]
model_list = []

M = 7
w = M
h = int(M/w)
pbc = False

for (n_idx, N) in enumerate(N_list): 
    O = N
    
    model = me.Bose_Hubbard(N, O, w=w, h=h, M=M, pbc=pbc)
    model_list.append(model)
    
print("The size of the Hamiltonian is {}".format(model.tot_states))

t = 1 
U_max = 5
V = 0
U_list_all = np.arange(1, U_max+1, .25)
#U_list_all =np.array([4])
mu_list_all = np.linspace(0, 10, 5)
mu_list_all = np.array([0.])


In [ ]:
from tqdm import tqdm

## NN

S = 50 # number of sample of the MH sampler (not used)
init = 1 # the first state to sample in Metropolis Hastings (has nothing to do with the optimizer!) (not used)

# Network parameters
D_hid = 400 # the number ofneurons in the hidden layer
lr = 5e-6 # learning rate 
#lr = 5e-6 # learning rate 
#lr = 1e-8 # learning rate
#epochs = 10000
epochs = 1000
loss_diff = 1e-7
grad_cut = 1e-6
check_point = 100 # print out the energy every X points
use_sampler = False # for now, only support ground state (not working anyway)

# Model parameters
U_train = np.ones(3)*2
t_train = 1.

U_train = np.logspace(-2, 2, 9, endpoint = True)

mu_train = np.zeros_like(U_train)
c1=0
c2=0
c3=1
qlist=[1, 2, 1000000.0,-1,-2]
#qlist=[-1]

n_avg_states=50
target_epsilon=0.5

min_state = 0
max_state = 1

n_excited = max_state - 1

# paths to save and load weights 
#fpath ='/storage/disqs/phrczh/HubbardNet/weights/'

#t0 = time.time()

loss_all = [] 
penalty_all = []
nn = []
energy_all = []
overlap_all = []
frac_dim_all0 = []
frac_dim_all1 = []
frac_dim_all2 = []
frac_dim_all3 = []
frac_dim_all4 = []
#frac_dim_all5 = []



# filepath for excited states
fname = "./weights/ep-1000-ADAM-weights_M{}N{}_V{}_Umax{}_Umin{}_L5_D{}-avgexcited_lr{}_len_Utrain{}_c1{}c2{}c3{}qlist{}n_avg_states{}target_epsilon{}".format(M,N,V,np.max(U_train),np.min(U_train),D_hid,lr,len(U_train),c1,c2,c3,qlist,n_avg_states,target_epsilon)


In [ ]:
t0 = time.time()
def call_NN(lr, n_excited):
    if n_excited: 
        if n_excited == 1 :
            load_states_indv = [0]
        else:
            load_states_indv = range(1, n_excited)
        gs_flag = False
        es_flag = True

    else: # ground states
        load_states_indv = [0]
        gs_flag = True
        es_flag = False
    
    load_states = np.max(load_states_indv)  # total number of states being fixed 
  
    params = {'D_hid': D_hid, 
              'step_size': lr, 
              'max_iteration':epochs,
              'check_point': check_point,
              'loss_diff': loss_diff, 
              'steps': 1000, # reset learning every N steps
              'loss_check_steps': 50, # check the local every N steps
              'grad_cut': grad_cut,  # stopping condition in the total gradient 
              'weight_init': False, 
              'zero_bias': False, 
              'gs_epochs': 1000, # the maximum number of steps to minimize the ground state
              'gs_flag': gs_flag, # ground state only
              'es_flag': es_flag,  # excited state only
              'regularization': True, 
              'load_states': load_states, # the number of states loaded 
              'load_states_indv': load_states_indv, 
              'rand_steps': 5000, 
              'load_weights_from_previous_state': False, # randomize the projection every N steps
              'use_gpu': use_gpu, 
              'weight_decay': 0,
              'perturb_amp': 0.00, 
              'dropout': 0.0}
  
    print("Begin optimizing for state {}".format(n_excited))

    #fc1, Loss_history, dot_history, all_E_list = train_NN(model_list, N_list, mu_train, U_train, t_train, V, S, params, fname, \
                                            #use_sampler=use_sampler, init=init, loadweights=False,\
                                            #fname_load=fname, n_excited=n_excited)
    fc1, Loss_history, energy_history, overlap_history, frac_dim_history0, frac_dim_history1, frac_dim_history2, frac_dim_history3, frac_dim_history4, dot_history, all_E_list = train_NN(model_list, N_list, mu_train, U_train, t_train, c1, c2, c3, qlist, V, S, params, fname, \
                                            use_sampler=use_sampler, init=init, loadweights=False,\
                                            fname_load=fname, n_excited=n_excited)

    #return fc1, Loss_history, dot_history
    return fc1, Loss_history, energy_history, overlap_history, frac_dim_history0, frac_dim_history1, frac_dim_history2, frac_dim_history3, frac_dim_history4, dot_history




for n_excited in range(min_state, max_state):
    
    fc1, Loss_history, energy_history, overlap_history, frac_dim_history0, frac_dim_history1, frac_dim_history2, frac_dim_history3, frac_dim_history4, dot_history = call_NN(lr, n_excited)

    nn.append(fc1)
    loss_all.append(Loss_history)
    energy_all.append(energy_history)
    overlap_all.append(overlap_history)
    penalty_all.append(dot_history)
    frac_dim_all0.append(frac_dim_history0)
    frac_dim_all1.append(frac_dim_history1)
    frac_dim_all2.append(frac_dim_history2)
    frac_dim_all3.append(frac_dim_history3)
    frac_dim_all4.append(frac_dim_history4)
    #frac_dim_all5.append(frac_dim_history5)
    
tf = time.time()
print("Training time = {} seconds.".format(tf-t0))
#np.savetxt(f'./time/M-{M}-N-{N}-U_train-{U_train}-lr-{lr}-num_th-{torch.get_num_threads()}-interop-{torch.get_num_interop_threads()}-use_gpu-{use_gpu}-adam.csv',[tf-t0])
np.savetxt(f'./time-excited-target/epoch-{epochs}-ADAM-excited-c1-{c1}-c2-{c2}-c3-{c3}-qlist-{qlist}-epsilon-{target_epsilon}-n_avg_states-{n_avg_states}-M-{M}-N-{N}-len_U_train-{len(U_train)}-lr-{lr}-num_th-{torch.get_num_threads()}-interop-{torch.get_num_interop_threads()}-use_gpu-{use_gpu}.csv',[tf-t0])

In [ ]:
U_test = np.logspace(-2.125, 2.125, 69, endpoint = True) 
#U_test = np.logspace(-2, 2, 121, endpoint = True) 
#U_test = np.logspace(-3, 2, 121, endpoint = True) 
#U_test =np.array([4])
mu_test = np.zeros_like(U_test)

fc1 = load_weights(fname + "gs", M, D_hid)
E_train, wf_gs = wf_e_calc(model_list[0], N_list[0], U_train, mu_train, t, V, 0, 0, fc1, use_gpu=True)
E_test, wf_gs_test = wf_e_calc(model_list[0], N_list[0], U_test, mu_test, t, V, 0, 0, fc1, use_gpu=True)
